# Estratégia de utilização de Bag Of N-gram com Rede Neural

## Imports e definição de funções

In [1]:
import os
import shutil
import time
import pickle
import pandas as pd
import statistics as stat
import numpy as np
import tensorflow as tf
import tflearn
from tflearn.layers.core import input_data, dropout, fully_connected
from tflearn.layers.conv import conv_1d, global_max_pool
from tflearn.layers.merge_ops import merge
from tflearn.layers.estimator import regression
from nltk.corpus import stopwords
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
# Função para salvar uma lista em um binário
def save_list_to_file(list_to_save,file_name):
    with open(file_name, 'wb') as f:
        pickle.dump(list_to_save, f)
# Função para carregar uma lista 
def load_list_from_file(file_name):
    with open(file_name, 'rb') as f:
        return pickle.load(f)     
# Criando a classe de parada antecipada
class EarlyStoppingCallback(tflearn.callbacks.Callback):
    def __init__(self, val_epoch_thresh, val_acc_thresh):
        self.val_epoch_thresh = val_epoch_thresh
        self.val_acc_thresh = val_acc_thresh
    def on_epoch_end(self, training_state):
        print("Epoch ", training_state.epoch, " with Accuracy ", training_state.acc_value)
        if training_state.epoch >= self.val_epoch_thresh and training_state.acc_value >= self.val_acc_thresh:
            raise StopIteration
    def on_train_end(self, training_state):
        print("Successfully left training! Final model accuracy:", training_state.acc_value)
print ('Importações e criação de funções realizadas com sucesso')

Importações e criação de funções realizadas com sucesso


## Restaurando o Pickle contendo o DataFrame trabalhado

In [2]:
# Restaurando o DataFrame gerado no Passo 1 - Preparação
FILENAME = 'dataframe.pickle'
df = pd.read_pickle(FILENAME)
df.head()

,sistema,classificacao,grupo,descricao_demanda,descricao,solucao,solucao_usuario,descricao_demanda2,solucao_usuario2
0,SNE,SNE,ATCDMGSNE,Solicitação de serviço: 2018SS/0000401273 Qual...,SNE,,"Prezado Senhor André, O Sistema de Notificação...",solicitaca serv proced agor,prez senh andr sistem notificaca eletron sne d...
1,SNE,SNE,UNCEDFSNE,Usuário informa que recebeu umas notificações ...,SNE,,"Sr. Usuário,\n\nO órgão autuador faz algumas v...",usuari inform receb uma notificaco aplic veicu...,usuari orga autu faz algum verificaco sab real...
2,SNE,SNE,CAGSBRSNE,Usuário informa que emitiu um boleto que vence...,SNE,,"Prezado Angelo,\n\nQual a data de vencimento d...",usuari inform emit bolet venc praz acab vend c...,prez angel dat venc infraca comunidad atend sne
3,SNE,SNE,CAGSBRSNE,Usuário informa que esta tentando cadastrar um...,SNE,,"Prezado Tiago,\nInformamos que o código de seg...",usuari inform tent cadastr veicul sne inser co...,prez tiag inform codig seguranc crvdut encontr...
4,SNE,SNE,CAGSBRSNE,Assunto: MULTA PAGA VOLTANDO A CONTAR NO SISTE...,SNE,2016SOL/00400157,"Prezado Jeferson,\n\n\nFavor nos enviar PLACA ...",assunt mult pag volt cont sistem sne jeferson ...,prez jeferson favor envi plac ren poss verific...


In [3]:
# Descrevendo os dados do DataFrame
df.describe()

,sistema,classificacao,grupo,descricao_demanda,descricao,solucao,solucao_usuario,descricao_demanda2,solucao_usuario2
count,6291,6291,6291,6291,6291,6291,6291,6291,6291
unique,7,19,20,5914,20,62,2056,5598,1944
top,SOLICITAÇÃO,DÚVIDA - INFORMAÇÃO,ATGSBRTERC,Usuário pergunta como pode se CADASTRAR no SIS...,"SNE,SOLICITAÇÃO,DÚVIDA - INFORMAÇÃO",,A consulta dos órgãos que fizeram adesão ao SN...,usuari pergunt pod cadastr sistem,prezado sra realiz cadastr soluca sra pod baix...
freq,1789,2041,2432,35,1789,1521,311,35,353


In [4]:
df.groupby('sistema').count()

,classificacao,grupo,descricao_demanda,descricao,solucao,solucao_usuario,descricao_demanda2,solucao_usuario2
sistema,,,,,,,,
ACESSO,420,420,420,420,420,420,420,420
APLICAÇÃO,1438,1438,1438,1438,1438,1438,1438,1438
CADASTRO,1060,1060,1060,1060,1060,1060,1060,1060
SNE,572,572,572,572,572,572,572,572
SNE - ATENDIMENTO LOJAS,915,915,915,915,915,915,915,915
SNE - MSG ERRO: OCORREU UM ERRO INESPERADO,97,97,97,97,97,97,97,97
SOLICITAÇÃO,1789,1789,1789,1789,1789,1789,1789,1789


## Criação do Bag Of N-gram para X

In [5]:
# Se essa linha der erro, reinicie o Jupyter com o comando abaixo:
#
#  jupyter notebook --NotebookApp.iopub_data_rate_limit=1.0e10
#
start_time = time.time()
MAX_FEATURES=15000
# Cria um ngram vectorizer
# Vamos criar o BOW dos 2 documentos
# Usamos n_gram = 2
vectorizer = CountVectorizer(ngram_range=(2,2), max_features=MAX_FEATURES)
#vectorizer = TfidfVectorizer(ngram_range=(2,2),stop_words=stopwords.words('portuguese'),strip_accents='unicode',max_features=MAX_FEATURES)
# Contabilizamos as combinações de caracteres nas palavras
X_vet = vectorizer.fit_transform(df['descricao_demanda2'])
print('Criação de X em %.2f segundos' % (time.time() - start_time))
print('Formato de X_vet: ',X_vet.shape)

Criação de X em 0.20 segundos
Formato de X_vet:  (6291, 15000)


In [6]:
# Imprimindo os valores para cada token no primeiro documento
for doc in X_vet:
    print('------------')
    scores = zip(vectorizer.get_feature_names(),
                 np.asarray(doc.sum(axis=0)).ravel())
    sorted_scores = sorted(scores, key=lambda x: x[1], reverse=True)
    for item in sorted_scores:
        if item[1] > 0.0:
            print ("{0:30} Score: {1}".format(item[0], item[1]))
    break

------------
proced agor                    Score: 1
solicitaca serv                Score: 1


## Criação do One Hot Encoding para X

In [7]:
# Preparando dados de X
start_time = time.time()
X_vocabulary = vectorizer.get_feature_names()
NUM_FEATURES = len(X_vocabulary)
print('Tamanho do vocabulário: {}'.format(len(X_vocabulary)))
print('Imprimindo os 100 primeiros:')
print(X_vocabulary[:100])
print('--------------------------------------------------------------------------------------------------------------')
X_values = X_vet.toarray().astype(float)
print('Formato de X {}:'.format(X_values.shape))
print('Imprimindo One-Hot Encoding dos 5 primeiros:')
print(X_values[:5])
print('Criação de Y em %.2f segundos' % (time.time() - start_time))

Tamanho do vocabulário: 15000
Imprimindo os 100 primeiros:
['aba veicul', 'abaix aind', 'abaix atend', 'abaix cadastr', 'abaix cpf', 'abaix descrev', 'abaix detalh', 'abaix encaminh', 'abaix err', 'abaix inform', 'abaix instal', 'abaix mens', 'abaix nao', 'abaix plac', 'abaix por', 'abaix precis', 'abaix resolv', 'abaix ter', 'abaix verific', 'abat val', 'abert acion', 'abert app', 'abert aqu', 'abert atend', 'abert cham', 'abert const', 'abert fac', 'abert pag', 'abert por', 'abert precis', 'abert sit', 'abert solicitaca', 'ablic agor', 'about review', 'abr acion', 'abr aplic', 'abr apo', 'abr app', 'abr cham', 'abr clic', 'abr faz', 'abr formulari', 'abr gmt', 'abr mao', 'abr nov', 'abr ocorrenc', 'abr outr', 'abr pagin', 'abr ped', 'abr protocol', 'abr solicitaca', 'abr tod', 'abrac vand', 'abril nao', 'abrirabr nov', 'absolut nad', 'absurd respond', 'acab baix', 'acab cadastr', 'acab envi', 'acab ger', 'acab nao', 'acab receb', 'acab ver', 'aceit adesa', 'aceit banc', 'aceit cadast

## Criação do One Hot Encoding para Y

In [8]:
# Preparando dados de Y
#Vetorização dos labels (Target)
Y_labels = pd.get_dummies(df['sistema'])
print('Quantidade de targets: {}'.format(len(Y_labels)))
print('Imprimindo os targets:')
print('Targets: ',Y_labels.columns)
print('--------------------------------------------------------------------------------------------------------------')
Y_values = pd.get_dummies(df['sistema']).values
print('Formato de Y {}:'.format(Y_values.shape))
print('Imprimindo One-Hot Encoding do primeiro Y:')
print(Y_values[0])

Quantidade de targets: 6291
Imprimindo os targets:
Targets:  Index(['ACESSO', 'APLICAÇÃO', 'CADASTRO', 'SNE', 'SNE - ATENDIMENTO LOJAS',
       'SNE - MSG ERRO: OCORREU UM ERRO INESPERADO', 'SOLICITAÇÃO'],
      dtype='object')
--------------------------------------------------------------------------------------------------------------
Formato de Y (6291, 7):
Imprimindo One-Hot Encoding do primeiro Y:
[0 0 0 1 0 0 0]


## Separação dos dados de treinamento e teste

In [9]:
# Separando dados de treinamento e testes
if 'X_values' in locals():
    X_train, X_test, y_train, y_test = train_test_split(X_values, Y_values, test_size=0.33, random_state=42)
    print(' Train data: X_train ',X_train.shape,', y_train',y_train.shape)
    print(' Test data: X_test ',X_test.shape,', y_test ',y_test.shape)
    print('Exemplo de X_train: ',X_train[0][:100],'...')
    print('Exemplo de y_train: ',y_train[0])
    del X_values, Y_values

 Train data: X_train  (4214, 15000) , y_train (4214, 7)
 Test data: X_test  (2077, 15000) , y_test  (2077, 7)
Exemplo de X_train:  [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0.] ...
Exemplo de y_train:  [0 0 1 0 0 0 0]


## Criação de uma Rede Neural DNN para ser treinada

In [10]:
# Imprimindo primeiro dado de treino
print('Entrada no formato (indice:valor)')
i = 0
for x in X_train[0]:
    if x != 0.0:
        print (i,":",x)
    i += 1
print('Resultado esperado: ',y_train[0])   

Entrada no formato (indice:valor)
1596 : 1.0
1661 : 1.0
2380 : 1.0
2827 : 1.0
3341 : 1.0
6582 : 1.0
10001 : 1.0
10676 : 1.0
11468 : 1.0
Resultado esperado:  [0 0 1 0 0 0 0]


In [11]:
DATASET = 'sne'
REDE = 'net_3'
BATCH = 32
EPOCHS = 100
MIN_EPOCHS = 5
MIN_ACCURACY = 0.9999
print ('Parâmetros setados')

Parâmetros setados


In [12]:
# Reset do grafo
tf.reset_default_graph()
# Cria a rede neural
net = None
if REDE == 'net1':
    net = tflearn.input_data(shape=[None, len(X_train[0])])
    net = tflearn.fully_connected(net, 8)
    net = tflearn.fully_connected(net, 8)
    net = tflearn.fully_connected(net, len(y_train[0]), activation='softmax')
    net = tflearn.regression(net)
elif REDE == 'net_1':
    net = tflearn.input_data(shape=[None, len(X_train[0])])
    net = tflearn.fully_connected(net, 8)
    net = tflearn.dropout(net, 0.8)
    net = tflearn.fully_connected(net, 8)
    net = tflearn.dropout(net, 0.8)
    net = tflearn.fully_connected(net, len(y_train[0]), activation='softmax')
    net = tflearn.regression(net)
elif REDE == 'net_2':
    net = tflearn.input_data(shape=[None, len(X_train[0])])
    net = tflearn.fully_connected(net, 32)
    net = tflearn.dropout(net, 0.8)
    net = tflearn.fully_connected(net, 16)
    net = tflearn.dropout(net, 0.8)
    net = tflearn.fully_connected(net, len(y_train[0]), activation='softmax')
    net = tflearn.regression(net)
elif REDE == 'net_3':
    net = tflearn.input_data(shape=[None, len(X_train[0])])
    net = tflearn.fully_connected(net, 32)
    net = tflearn.dropout(net, 0.5)
    net = tflearn.fully_connected(net, 16)
    net = tflearn.dropout(net, 0.5)
    net = tflearn.fully_connected(net, len(y_train[0]), activation='softmax')
    net = tflearn.regression(net)
elif REDE == 'net_3_1':
    net = tflearn.input_data(shape=[None, len(X_train[0])])
    net = tflearn.fully_connected(net, 32)
    net = tflearn.dropout(net, 0.4)
    net = tflearn.fully_connected(net, 16)
    net = tflearn.dropout(net, 0.4)
    net = tflearn.fully_connected(net, len(y_train[0]), activation='softmax')
    net = tflearn.regression(net)
elif REDE == 'net_3_2':
    net = tflearn.input_data(shape=[None, len(X_train[0])])
    net = tflearn.fully_connected(net, 32)
    net = tflearn.dropout(net, 0.3)
    net = tflearn.fully_connected(net, 16)
    net = tflearn.dropout(net, 0.3)
    net = tflearn.fully_connected(net, len(y_train[0]), activation='softmax')
    net = tflearn.regression(net)
elif REDE == 'net_4':
    net = tflearn.input_data(shape=[None, len(X_train[0])])
    net = tflearn.fully_connected(net, 16)
    net = tflearn.dropout(net, 0.5)
    net = tflearn.fully_connected(net, 8)
    net = tflearn.dropout(net, 0.5)
    net = tflearn.fully_connected(net, len(y_train[0]), activation='softmax')
    net = tflearn.regression(net)
elif REDE == 'net_5':
    net = tflearn.input_data(shape=[None, len(X_train[0])])
    net = tflearn.fully_connected(net, 32)
    net = tflearn.dropout(net, 0.5)
    net = tflearn.fully_connected(net, 16)
    net = tflearn.dropout(net, 0.5)
    net = tflearn.fully_connected(net, 8)
    net = tflearn.dropout(net, 0.5)
    net = tflearn.fully_connected(net, len(y_train[0]), activation='softmax')
    net = tflearn.regression(net)
elif REDE == 'net_6':
    net = tflearn.input_data(shape=[None, len(X_train[0])])
    net = tflearn.fully_connected(net, 128)
    net = tflearn.dropout(net, 0.5)
    net = tflearn.fully_connected(net, 64)
    net = tflearn.dropout(net, 0.5)
    net = tflearn.fully_connected(net, 32)
    net = tflearn.dropout(net, 0.5)
    net = tflearn.fully_connected(net, len(y_train[0]), activation='softmax')
    net = tflearn.regression(net)
elif REDE == 'net_cnn_1':
    net = input_data(shape=[None, X_train.shape[1]], name='input')
    net = tflearn.embedding(net, input_dim=NUM_FEATURES, output_dim=128)
    branch1 = conv_1d(net, 32, 3, padding='same', activation='relu', regularizer="L2")
    branch2 = conv_1d(net, 32, 5, padding='same', activation='relu', regularizer="L2")
    net = merge([branch1, branch2], mode='concat', axis=1)
    net = tf.expand_dims(net, 2)
    net = global_max_pool(net)
    net = dropout(net, 0.5)
    net = fully_connected(net, len(y_train[0]), activation='softmax')
    net = regression(net, optimizer='adam', learning_rate=0.001,
                     loss='categorical_crossentropy', name='target')
elif REDE == 'net_8':
    net = input_data(shape=[None, X_train.shape[1]], name='input')
    net = tflearn.embedding(net, input_dim=NUM_FEATURES, output_dim=128)
    branch1 = conv_1d(net, 128, 3, padding='same', activation='relu', regularizer="L2")
    branch2 = conv_1d(net, 128, 5, padding='same', activation='relu', regularizer="L2")
    branch3 = conv_1d(net, 128, 7, padding='same', activation='relu', regularizer="L2")
    net = merge([branch1, branch2, branch3], mode='concat', axis=1)
    net = tf.expand_dims(net, 2)
    net = global_max_pool(net)
    net = dropout(net, 0.5)
    net = fully_connected(net, len(y_train[0]), activation='softmax')
    net = regression(net, optimizer='adam', learning_rate=0.001,
                     loss='categorical_crossentropy', name='target')
elif REDE == 'net2':
    net = tflearn.input_data(shape=[None, len(X_train[0])])
    net = tflearn.fully_connected(net, 16)
    net = tflearn.fully_connected(net, 16)
    net = tflearn.fully_connected(net, len(y_train[0]), activation='softmax')
    net = tflearn.regression(net)
elif REDE == 'net3':
    net = tflearn.input_data(shape=[None, len(X_train[0])])
    net = tflearn.fully_connected(net, 8)
    net = tflearn.fully_connected(net, 8)
    net = tflearn.fully_connected(net, 8)
    net = tflearn.fully_connected(net, len(y_train[0]), activation='softmax')
    net = tflearn.regression(net)
elif REDE == 'net4':
    net = tflearn.input_data(shape=[None, len(X_train[0])])
    net = tflearn.fully_connected(net, 100)
    net = tflearn.fully_connected(net, 50)
    net = tflearn.fully_connected(net, 25)
    net = tflearn.fully_connected(net, len(y_train[0]), activation='softmax')
    net = tflearn.regression(net)
elif REDE == 'net5':
    net = tflearn.input_data(shape=[None, len(X_train[0])])
    net = tflearn.embedding(net, input_dim=len(X_train[0]), output_dim=len(y_train[0]))
    net = tflearn.lstm(net, 8)
    net = tflearn.dropout(net, 0.8)
    net = tflearn.fully_connected(net, len(y_train[0]), activation='softmax')
    net = tflearn.regression(net, optimizer='adam',loss='binary_crossentropy')
elif REDE == 'net6':
    net = tflearn.input_data(shape=[None, len(X_train[0])])
    net = tflearn.embedding(net, input_dim=len(X_train[0]), output_dim=len(y_train[0]))
    net = tflearn.lstm(net, 8)
    net = tflearn.dropout(net, 0.8)
    net = tflearn.fully_connected(net, len(y_train[0]), activation='softmax')
    net = tflearn.regression(net, optimizer='adam',loss='binary_crossentropy')
# Define o modelo e configura o tensorboard
print ('Criada a',REDE)
model = tflearn.DNN(net, tensorboard_verbose=3, tensorboard_dir='/home/03662232677/c4/PUCOMEX/tflearn_logs_'+str(DATASET)+'/'+str(DATASET)+'_n'+str(NUM_FEATURES)+'_st'+str(EPOCHS)+'_'+str(REDE)+'_b'+str(BATCH))

Instructions for updating:
keep_dims is deprecated, use keepdims instead
Criada a net_3


## Treinamento da Rede Neural com os dados de treino

In [13]:
# Iniciando o treinamento da rede neural. Acompanhe o treinamento pelo TensorBoard utilizando o comando abaixo via terminal:
#
# tensorboard --logdir tflearn_logs_...
# 
start_time = time.time()
early_stopping_cb = EarlyStoppingCallback(val_epoch_thresh=MIN_EPOCHS,val_acc_thresh=MIN_ACCURACY)
# Treinamento
try:
    model.fit(X_train, y_train, n_epoch = EPOCHS, batch_size = BATCH, show_metric = True, callbacks=early_stopping_cb)
except StopIteration:
    print("Caught callback exception. Returning control to user program.")
print('Finalização do treinamento em %.2f segundos' % (time.time() - start_time))

Training Step: 13199  | total loss: 0.14938 | time: 2.803s
| Adam | epoch: 100 | loss: 0.14938 - acc: 0.9539 -- iter: 4192/4214
Training Step: 13200  | total loss: 0.14029 | time: 2.825s
| Adam | epoch: 100 | loss: 0.14029 - acc: 0.9585 -- iter: 4214/4214
--
Epoch  100  with Accuracy  0.9585309028625488
Successfully left training! Final model accuracy: 0.9585309028625488
Finalização do treinamento em 293.00 segundos


## Calculando a acurácia

In [14]:
# Salvando o modelo
#model.save('/home/03662232677/c4/PUCOMEX/model/'+str(DATASET)+'_n'+str(NUM_FEATURES)+'_st'+str(EPOCHS)+'_'+str(REDE)+'_b'+str(BATCH)+'.tflearn')
# Restaurando o modelo
#model.load('/home/03662232677/c4/PUCOMEX/model/'+str(DATASET)+'_n'+str(NUM_FEATURES)+'_st'+str(EPOCHS)+'_'+str(REDE)+'_b'+str(BATCH)+'.tflearn')

In [15]:
# Testando o modelo com a base de testes
total = 0
acertos = 0
score1 = model.evaluate(X_test, y_test)
print('Acurácia para os dados de teste: {}'.format(score1[0]))
score2 = model.evaluate(X_train, y_train)
print('Acurácia para os dados de treinamento: {}'.format(score2))
print('Acurácia para todos os dados: {}'.format(stat.mean(score1+score2)))

Acurácia para os dados de teste: 0.5642753970783723
Acurácia para os dados de treinamento: [0.9575225445519254]
Acurácia para todos os dados: 0.7608989708151488


## Realizando um teste de um documento

In [88]:
N = 3
#print(X_test[N])
print(np.rint(model.predict([X_test[N]]))[0])
print(y_test[N])

[0. 0. 1. 0. 0.]
[0 0 0 1 0]


## Salvando um modelo treinado

In [89]:
# Save model and pickle files
model_name = 'model_'+str(DATASET)+'_n'+str(MAX_FEATURES)+'_st'+str(EPOCHS)+'_'+str(REDE)+'_b'+str(BATCH)
directory = '/home/03662232677/c4/PUCOMEX/'+model_name
if not os.path.exists(directory):
    os.mkdir(directory) 
model.save(directory+'/'+model_name+'.tflearn')
save_list_to_file(X_vocabulary,directory+'/X_vocabulary.pickle')
#save_list_to_file(X_values,directory+'/X_values.pickle')
#save_list_to_file(Y_labels,directory+'/Y_labels.pickle')
#save_list_to_file(Y_values,directory+'/Y_values.pickle')
save_list_to_file(X_train,directory+'/X_train.pickle')
save_list_to_file(X_test,directory+'/X_test.pickle')
save_list_to_file(y_train,directory+'/y_train.pickle')
save_list_to_file(y_test,directory+'/y_test.pickle')

INFO:tensorflow:/home/03662232677/c4/PUCOMEX/model_pucomex_n15000_st100_net_3_b32/model_pucomex_n15000_st100_net_3_b32.tflearn is not in all_model_checkpoint_paths. Manually adding it.


FIM

In [90]:
# Restaurando o DataFrame2
FILENAME2 = 'dataframe2.pickle'
df2 = pd.read_pickle(FILENAME2)
df2.groupby('classificacao').count()

,sistema,grupo,descricao_demanda,descricao,solucao,solucao_usuario,solucao_script,descricao_demanda2,solucao_usuario2,solucao_script2,classificacao_grupo
classificacao,,,,,,,,,,,
SITUAÇÃO NÃO PREVISTA,4071,4071,4071,4071,4071,4071,4071,4071,4071,4071,4071


In [91]:
# Selecionando apenas as colunas desejadas
df2 = df2[['sistema','classificacao','grupo','descricao_demanda','descricao_demanda2']]
df2.head()

,sistema,classificacao,grupo,descricao_demanda,descricao_demanda2
10,DUEX - DECLARAÇÃO ÚNICA DE EXPORTAÇÃO,SITUAÇÃO NÃO PREVISTA,CAGSBRPUCOMEX,"Bom dia, ao tentarmos retificar a Due no porta...",bom dia tent retific due port receb err inform...
12,DUEX - DECLARAÇÃO ÚNICA DE EXPORTAÇÃO,SITUAÇÃO NÃO PREVISTA,CAGSBRPUCOMEX,"Sem acesso a criação de DU-E, opção de Embarqu...",acess criaca due opca embarqu antecip vinculac...
23,DUEX - DECLARAÇÃO ÚNICA DE EXPORTAÇÃO,SITUAÇÃO NÃO PREVISTA,CAGSBRPUCOMEX,"Bom dia, Conforme contato telefônico no qual f...",bom dia conform contat telefon ger protocol se...
26,DUEX - DECLARAÇÃO ÚNICA DE EXPORTAÇÃO,SITUAÇÃO NÃO PREVISTA,CAGSBRPUCOMEX,O sistema está passando por uma indisponibilid...,sistem pass indisponibil momentane possi agu a...
27,DUEX - DECLARAÇÃO ÚNICA DE EXPORTAÇÃO,SITUAÇÃO NÃO PREVISTA,CAGSBRNOVOEX,Erro ao consultar DUE,err consult due


In [92]:
# Imprime as colunas de labels
print(Y_labels.columns)

Index(['AMBIENTE', 'ANÁLISE URC', 'APLICAÇÃO', 'DÚVIDA OU INFORMAÇÕES',
       'REGRA DE NEGÓCIO'],
      dtype='object')


In [93]:
# Cria uma nova coluna com valores vazios
df2['resultado'] = ''
# Para cada valor, faz a previsão e grava o resultado na coluna
c = 0
print(df2.columns)
for row_index, row in df2.iterrows():
    #print(row_index)
    # Texto pré-processado a ser utilizado
    t = row['descricao_demanda2']
    #print(t)
    # Vetorização do texto
    x = vectorizer.transform([t])
    # Transformando para array
    array = x.toarray()
    #for xi in array[0]:
    #    if xi > 0:
    #        print (xi)
    # Fazendo a previsão
    result = model.predict(array)
    # Buscando o label correspondente
    label = Y_labels.columns[np.argmax(result)]
    #print (label)
    # Salvando na coluna de resultado
    df2.at[row_index,'resultado'] = label
    c += 1
    if c % 1000 == 0:
        print(c,'itens processados')
print('total de',c,'itens processados')

Index(['sistema', 'classificacao', 'grupo', 'descricao_demanda',
       'descricao_demanda2', 'resultado'],
      dtype='object')
1000 itens processados
2000 itens processados
3000 itens processados
4000 itens processados
total de 4071 itens processados


In [94]:
df2.head()

,sistema,classificacao,grupo,descricao_demanda,descricao_demanda2,resultado
10,DUEX - DECLARAÇÃO ÚNICA DE EXPORTAÇÃO,SITUAÇÃO NÃO PREVISTA,CAGSBRPUCOMEX,"Bom dia, ao tentarmos retificar a Due no porta...",bom dia tent retific due port receb err inform...,DÚVIDA OU INFORMAÇÕES
12,DUEX - DECLARAÇÃO ÚNICA DE EXPORTAÇÃO,SITUAÇÃO NÃO PREVISTA,CAGSBRPUCOMEX,"Sem acesso a criação de DU-E, opção de Embarqu...",acess criaca due opca embarqu antecip vinculac...,DÚVIDA OU INFORMAÇÕES
23,DUEX - DECLARAÇÃO ÚNICA DE EXPORTAÇÃO,SITUAÇÃO NÃO PREVISTA,CAGSBRPUCOMEX,"Bom dia, Conforme contato telefônico no qual f...",bom dia conform contat telefon ger protocol se...,APLICAÇÃO
26,DUEX - DECLARAÇÃO ÚNICA DE EXPORTAÇÃO,SITUAÇÃO NÃO PREVISTA,CAGSBRPUCOMEX,O sistema está passando por uma indisponibilid...,sistem pass indisponibil momentane possi agu a...,AMBIENTE
27,DUEX - DECLARAÇÃO ÚNICA DE EXPORTAÇÃO,SITUAÇÃO NÃO PREVISTA,CAGSBRNOVOEX,Erro ao consultar DUE,err consult due,DÚVIDA OU INFORMAÇÕES


In [95]:
#!pip install openpyxl
# Salvando para formato Excel
writer = pd.ExcelWriter('pucomex_situacao_nao_prevista.xlsx')
df2.to_excel(writer,'pucomex')
writer.save()
print('Arquivo salvo com sucesso!')

Arquivo salvo com sucesso!


FIM